In [ ]:
# !pip install -U transformers tokenizers pandas torch accelerate datasets evaluate

  Using cached transformers-4.56.0-py3-none-any.whl.metadata (40 kB)
  Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.8 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.56.0-py3-none-any.whl (11.6 MB)
Using cached tokenizers-0.22.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.21.0
    Uninstalling datasets-2.21.0:
      Successfully uninstalled datasets-2.21.0


In [ ]:
# !pip install -U transformers==4.44.2 datasets==2.21.0 evaluate accelerate

  Using cached transformers-4.44.2-py3-none-any.whl.metadata (43 kB)
  Using cached datasets-2.21.0-py3-none-any.whl.metadata (21 kB)
  Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.44.2-py3-none-any.whl (9.5 MB)
Using cached datasets-2.21.0-py3-none-any.whl (527 kB)
Using cached tokenizers-0.19.1-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.0
    Uninstalling tokenizers-0.22.0:
      Successfully uninstalled tokenizers-0.22.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.0
    Uninstalling transformers-4.56.0:
      Successfully uninstalled transformers-4.56.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [ ]:
# !pip install -q transformers==4.44.2 datasets==2.21.0 evaluate accelerate

In [ ]:
!pip install -q transformers==4.44.2 datasets==2.21.0 evaluate accelerate

In [ ]:
import json, pandas as pd
from pathlib import Path
from datasets import Dataset
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer, EarlyStoppingCallback, AutoTokenizer, pipeline
import evaluate
import numpy as np
import torch, platform
import time

In [ ]:
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("Capability:", torch.cuda.get_device_capability(0))
print("Python:", platform.python_version())

CUDA available: False
Python: 3.12.11


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data Preparation

In [ ]:
data_dir = Path("/content/drive/MyDrive/")
train_path = data_dir / "train-v1.1.json"
dev_path   = data_dir / "dev-v1.1.json"

def load_squad_json(path):
    with open(path, "r", encoding="utf-8") as f:
        js = json.load(f)
    rows = []
    for art in js["data"]:
        title = art.get("title", "")
        for para in art["paragraphs"]:
            context = para["context"]
            for qa in para["qas"]:
                qid = qa["id"]
                question = qa["question"]
                answers = qa["answers"]
                ans_texts = [a["text"] for a in answers]
                ans_starts = [a["answer_start"] for a in answers]
                rows.append({
                    "id": qid,
                    "title": title,
                    "context": context,
                    "question": question,
                    "answers_text": ans_texts,
                    "answers_start": ans_starts,
                })
    return pd.DataFrame(rows)

train_df = load_squad_json(train_path)
dev_df   = load_squad_json(dev_path)

print("Train shape:", train_df.shape)
print("Dev shape:", dev_df.shape)
print("\nTrain sample:")
print(train_df.head(2)[["id","title","question","answers_text"]])

Train shape: (87599, 6)
Dev shape: (10570, 6)

Train sample:
                         id                     title  \
0  5733be284776f41900661182  University_of_Notre_Dame   
1  5733be284776f4190066117f  University_of_Notre_Dame   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   

                   answers_text  
0  [Saint Bernadette Soubirous]  
1   [a copper statue of Christ]  


In [ ]:
for i in range(3):
    row = train_df.iloc[i]
    ctx = row["context"]
    text = row["answers_text"][0]
    start = row["answers_start"][0]
    print(f"Q: {row['question']}")
    print(f"A (from DataFrame): {text}")
    print(f"A (from Context):   {ctx[start:start+len(text)]}")
    print("\n")

Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
A (from DataFrame): Saint Bernadette Soubirous
A (from Context):   Saint Bernadette Soubirous


Q: What is in front of the Notre Dame Main Building?
A (from DataFrame): a copper statue of Christ
A (from Context):   a copper statue of Christ


Q: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
A (from DataFrame): the Main Building
A (from Context):   the Main Building




# Tokenization & Feature Preparation

In [ ]:
model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
max_length = 256
doc_stride = 96

def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    tokenized = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = tokenized.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized.pop("offset_mapping")

    start_positions, end_positions = [], []
    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized.sequence_ids(i)
        sample_idx = sample_mapping[i]
        answers = examples["answers_text"][sample_idx]
        starts   = examples["answers_start"][sample_idx]

        if len(answers) == 0:
            start_positions.append(cls_index); end_positions.append(cls_index); continue

        start_char = starts[0]; end_char = start_char + len(answers[0])

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        token_end_index = len(input_ids) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
            start_positions.append(cls_index); end_positions.append(cls_index)
        else:
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)
            while offsets[token_end_index][1] >= end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized["start_positions"] = start_positions
    tokenized["end_positions"] = end_positions
    return tokenized


train_dataset = Dataset.from_pandas(train_df)
dev_dataset   = Dataset.from_pandas(dev_df)

tokenized_train = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_dev   = dev_dataset.map(preprocess_function,   batched=True, remove_columns=dev_dataset.column_names)

print(tokenized_train)
print(tokenized_dev)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 96847
})
Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 11966
})


In [ ]:
raw_example = train_dataset[0]
processed_example = preprocess_function({
    "question": [raw_example["question"]],
    "context": [raw_example["context"]],
    "answers_text": [raw_example["answers_text"]],
    "answers_start": [raw_example["answers_start"]],
})
input_ids = processed_example["input_ids"][0]
start = processed_example["start_positions"][0]
end = processed_example["end_positions"][0]

decoded_text = tokenizer.decode(input_ids, skip_special_tokens=True)
answer_text = tokenizer.decode(input_ids[start:end+1])

print("Question:", raw_example["question"])
print("Gold Answer:", raw_example["answers_text"][0])
print("Decoded Answer:", answer_text)

Question: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Gold Answer: Saint Bernadette Soubirous
Decoded Answer: saint bernadette soubirous


# Model Setup

In [ ]:
TRAIN_SAMPLES = 2000
DEV_SAMPLES   = 500

small_train = tokenized_train.shuffle(seed=42).select(range(min(TRAIN_SAMPLES, len(tokenized_train))))
small_dev   = tokenized_dev.shuffle(seed=42).select(range(min(DEV_SAMPLES, len(tokenized_dev))))

print("Using", len(small_train), "train examples and", len(small_dev), "dev examples for quick run.")

Using 2000 train examples and 500 dev examples for quick run.


In [ ]:
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
batch_size = 4

args = TrainingArguments(
    output_dir="distilbert-qa-quick",
    evaluation_strategy="no",
    save_strategy="no",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    weight_decay=0.0,
    logging_steps=100,
    save_total_limit=1,
    load_best_model_at_end=False,
    fp16=False,
    report_to="none",
    dataloader_num_workers=2
)

/usr/local/lib/python3.12/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=small_train,
    eval_dataset=None,
    tokenizer=tokenizer,
)

start = time.time()
trainer.train()
end = time.time()
print(f"Quick training finished in { (end-start)/60:.2f} minutes")

Step,Training Loss
100,2.573800
200,2.250600


Quick training finished in 52.35 minutes


In [ ]:
print("Evaluating (loss only) on small dev set:")
eval_res = trainer.evaluate(eval_dataset=small_dev)
print(eval_res)

Evaluating (loss only) on small dev set:


{'eval_loss': 2.50173020362854, 'eval_runtime': 246.9917, 'eval_samples_per_second': 2.024, 'eval_steps_per_second': 0.255, 'epoch': 1.0}


In [ ]:
trainer.save_model("distilbert-qa-quick-model")
print("Saved quick model to distilbert-qa-quick-model/")

Saved quick model to distilbert-qa-quick-model/


# Evaluation

In [ ]:
model_path = "distilbert-qa-quick-model"
qa_pipeline = pipeline("question-answering", model=model_path, tokenizer=tokenizer)

In [ ]:
squad_metric = evaluate.load("squad")

In [ ]:
EVAL_SAMPLES = 200
small_dev = dev_df.sample(n=EVAL_SAMPLES, random_state=42)

preds, refs = [], []

for i, row in small_dev.iterrows():
    q = row["question"]
    ctx = row["context"]
    gold_answers = row["answers_text"]

    try:
        pred = qa_pipeline({"question": q, "context": ctx}, max_answer_len=30)
        pred_text = pred["answer"]
    except:
        pred_text = ""

    preds.append({"id": str(i), "prediction_text": pred_text})
    refs.append({"id": str(i), "answers": {"text": gold_answers, "answer_start": row["answers_start"]}})

In [ ]:
metrics = squad_metric.compute(predictions=preds, references=refs)

print("Evaluation on small dev set:")
print(metrics)

Evaluation on small dev set:
{'exact_match': 41.0, 'f1': 54.919567066733244}


# Professional Touch: Experimentation

In [ ]:
qa_pipeline = pipeline("question-answering", model="distilbert-qa-quick-model", tokenizer=tokenizer)

In [ ]:
context = """
The Nile River is the longest river in the world, flowing through northeastern Africa.
It has been the lifeblood of civilizations in Egypt for thousands of years.
The river is about 6,650 kilometers long and flows into the Mediterranean Sea.
"""

In [ ]:
questions = [
    "What is the longest river in the world?",
    "Where does the Nile River flow into?",
    "How long is the Nile River?"
]

for q in questions:
    result = qa_pipeline({"question": q, "context": context}, max_answer_len=30)
    print(f"Q: {q}")
    print(f"A: {result['answer']}\n")

Q: What is the longest river in the world?
A: northeastern Africa

Q: Where does the Nile River flow into?
A: northeastern Africa

Q: How long is the Nile River?
A: 6,650 kilometers

